# Students' Budget Planning Tool

In [57]:
import gradio as gr
import mysql.connector
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from decimal import Decimal
import pandas as pd

* Database Handler (Parent Class)

In [59]:
# Database Handler (Parent Class)
class DatabaseHandler:
    def __init__(self):
        try:
            self.db = mysql.connector.connect(
                host="localhost",
                user="root",
                password="0hnmarNyunt",
                database="student_budget_database"
            )
            self.cursor = self.db.cursor()
        except mysql.connector.Error as err:
            print(f"Database connection error: {err}")
            raise

    def execute_query(self, query, params=None):
        try:
            self.cursor.execute(query, params or ())
            self.db.commit()
            return self.cursor
        except mysql.connector.Error as err:
            print(f"Database error: {err}")
            return None


* Budget Tracker (Child Class) inherited from DatabaseHandler Part Class

In [61]:
# Budget Tracker (Child Class)
class BudgetTracker(DatabaseHandler):
    def __init__(self):
        super().__init__()

    # Save Income & Expense transactions into MySQL
    def add_transaction(self, account_type, amount, category, trans_datetime, trans_type, notes):
        query = """INSERT INTO transactions 
                   (account_type, amount, category, trans_datetime, trans_type, notes)
                   VALUES (%s, %s, %s, %s, %s, %s)"""
        return self.execute_query(query, (account_type, amount, category, trans_datetime, trans_type, notes))

    # Save Income & Expense Budget into MySQL
    def add_budget(self, estimated_percentage, estimated_amount, category, estimated_datetime, trans_type, notes):
        if trans_type == 'Expense':
            income_budget = self.get_monthly_income_budget()
            percentage = Decimal(str(estimated_percentage)) / Decimal(100)
            estimated_amount = percentage * income_budget
        query = """INSERT INTO budget 
                   (estimated_percentage, estimated_amount, category, estimated_datetime, trans_type, notes)
                   VALUES (%s, %s, %s, %s, %s, %s)"""
        return self.execute_query(query, (estimated_percentage, estimated_amount, category, estimated_datetime, trans_type, notes))
    
    # Get total monthly income budget from MySQL
    def get_monthly_income_budget(self):
        query = """SELECT SUM(estimated_amount) 
                   FROM budget 
                   WHERE trans_type = 'Income'
                   AND MONTH(estimated_datetime) = MONTH(CURRENT_DATE())
                   AND YEAR(estimated_datetime) = YEAR(CURRENT_DATE())"""
        self.cursor.execute(query)
        result = self.cursor.fetchone()
        return Decimal(result[0]) if result[0] else Decimal(0)

    # Get weekly transactions from MySQL
    def get_weekly_report(self):
        try:
            # Get transactions
            query = """SELECT * FROM transactions 
                       WHERE trans_datetime >= DATE_SUB(NOW(), INTERVAL 7 DAY)
                       ORDER BY trans_datetime DESC"""
            self.cursor.execute(query)
            transactions = self.cursor.fetchall()
            
            # Calculate overall balance
            balance_query = """SELECT 
                               SUM(CASE WHEN trans_type = 'Income' THEN amount ELSE 0 END) as total_income,
                               SUM(CASE WHEN trans_type = 'Expense' THEN amount ELSE 0 END) as total_expense,
                               SUM(CASE WHEN trans_type = 'Income' THEN amount ELSE -amount END) as total_balance
                               FROM transactions
                               WHERE trans_datetime >= DATE_SUB(NOW(), INTERVAL 7 DAY)"""
            self.cursor.execute(balance_query)
            income, expense, balance = self.cursor.fetchone()
            
            # Calculate balance by account type
            account_balance_query = """SELECT 
                                       account_type,
                                       SUM(CASE WHEN trans_type = 'Income' THEN amount ELSE 0 END) as income,
                                       SUM(CASE WHEN trans_type = 'Expense' THEN amount ELSE 0 END) as expense,
                                       SUM(CASE WHEN trans_type = 'Income' THEN amount ELSE -amount END) as balance
                                       FROM transactions
                                       WHERE trans_datetime >= DATE_SUB(NOW(), INTERVAL 7 DAY)
                                       GROUP BY account_type"""
            self.cursor.execute(account_balance_query)
            account_balances = self.cursor.fetchall()
            
            # Format results
            columns = ["ID", "Account", "Amount", "Category", "Date", "Type", "Notes"]
            df = pd.DataFrame(transactions, columns=columns)
            
            # Create summary DataFrame
            summary_data = [{"Account": "Weekly Summary", "Income": income, "Expense": expense, "Balance": balance}]
            for account_balance in account_balances:
                summary_data.append({
                    "Account": account_balance[0],
                    "Income": account_balance[1],
                    "Expense": account_balance[2],
                    "Balance": account_balance[3]
                })
            summary = pd.DataFrame(summary_data)
            print(summary)
            
            return {"transactions": df, "summary": summary}
        except mysql.connector.Error as err:
            print(f"Database error: {err}")
            return {"error": str(err)}
        except Exception as e:
            print(f"Error: {str(e)}")
            return {"error": str(e)}

    # Get monthly transactions to generate the pie chart
    def get_monthly_report(self):
        try:
            query = """SELECT category, SUM(amount), trans_type 
                       FROM transactions 
                       WHERE MONTH(trans_datetime) = MONTH(CURRENT_DATE())
                       AND YEAR(trans_datetime) = YEAR(CURRENT_DATE())
                       GROUP BY category, trans_type
                       ORDER BY category"""
            self.cursor.execute(query)
            return self.cursor.fetchall()
        except Exception as e:
            print(f"Error: {str(e)}")
            return []


    # Join two tables to find the monthly difference amount between actual and budget amount according to estimated %
    def get_monthly_estimated_comparison(self):
        query = """SELECT 
                   b.category,
                   b.estimated_percentage,
                   b.estimated_amount,
                   COALESCE(SUM(t.amount), 0) as actual_amount
                   FROM budget b
                   LEFT JOIN transactions t 
                   ON b.category = t.category
                   AND MONTH(b.estimated_datetime) = MONTH(t.trans_datetime)
                   AND YEAR(b.estimated_datetime) = YEAR(t.trans_datetime)
                   WHERE MONTH(b.estimated_datetime) = MONTH(CURRENT_DATE())
                   AND YEAR(b.estimated_datetime) = YEAR(CURRENT_DATE())
                   AND b.trans_type = 'Expense'
                   GROUP BY b.category, b.estimated_percentage, b.estimated_amount
                   ORDER BY b.category"""
        self.cursor.execute(query)
        return self.cursor.fetchall()

* GUI and Report Generation - Preparing data for interface

In [63]:
# GUI and Report Generation
def create_pie_chart(data, title):
    if not data:
        return None  # Return None if no data
    
    categories = [row[0] for row in data]
    amounts = [row[1] for row in data]
    
    plt.figure(figsize=(10,6))
    plt.pie(amounts, labels=amounts, autopct='%1.1f%%', startangle=90)
    #plt.pie(amounts, labels=categories, autopct=lambda p: f'{p*sum(amounts)/100:,.2f}', startangle=90)
    plt.title(title)
    plt.legend(categories)
    return plt.gcf()

def create_bar_chart(data):
    categories = [row[0] for row in data]
    budget_amounts = [row[1] for row in data]
    actual_amounts = [row[2] for row in data]
    differences = [budget - actual for budget, actual in zip(budget_amounts, actual_amounts)]

    x = range(len(categories))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 6))
    rects1 = ax.bar(x, budget_amounts, width, label='Budget')
    rects2 = ax.bar([i + width for i in x], actual_amounts, width, label='Actual')

    ax.set_ylabel('Amount')
    ax.set_title('Monthly Budget vs Actual Spending')
    ax.set_xticks([i + width/2 for i in x])
    ax.set_xticklabels(categories, rotation=45, ha='right')
    ax.legend()

    for i, diff in enumerate(differences):
        ax.text(i + width/2, max(budget_amounts[i], actual_amounts[i]), f'Diff: {diff:,.2f}', 
                ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    return fig
    
def create_estimated_bar_chart(data):
    categories = [row[0] for row in data]
    percentages = [row[1] for row in data]
    budget_amounts = [row[2] for row in data]
    actual_amounts = [row[3] for row in data]

    x = range(len(categories))
    width = 0.25

    fig, ax1 = plt.subplots(figsize=(14, 7))
    
    # Bar plots
    ax1.bar(x, budget_amounts, width, label='Budgeted Expense') #, color='#ff9999'
    ax1.bar([i + width for i in x], actual_amounts, width, label='Actual Expense') #, color='#66b3ff'
    
    # Percentage line plot
    ax2 = ax1.twinx()
    ax2.plot([i + width/2 for i in x], percentages, 'ko--', label='Budget %')
    
    # Labels and formatting
    ax1.set_ylabel('Amount') #, color='#333333'
    ax2.set_ylabel('Percentage (%)') #, color='#333333'
    ax1.set_title('Monthly Expense: Budget vs Actual with Percentage') #, pad=20
    ax1.set_xticks([i + width/2 for i in x])
    ax1.set_xticklabels(categories, rotation=45, ha='right')
    
    # Value annotations
    for i, (budget, actual, pct) in enumerate(zip(budget_amounts, actual_amounts, percentages)):
        ax1.text(i + width/2, max(budget, actual), 
                f'Δ: {budget-actual:,.2f}\n{pct}%', 
                ha='center', va='bottom', fontsize=8)

    # Combined legend
    lines, labels = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines + lines2, labels + labels2) # , loc='upper left'

    plt.tight_layout()
    return fig


# Gradio Interface Functions
def add_income_transaction(account_type, amount, category, trans_datetime, notes):
    try:
        # Convert timestamp to MySQL datetime format
        if isinstance(trans_datetime, (int, float)):
            trans_datetime = datetime.fromtimestamp(trans_datetime).strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(trans_datetime, str):
            trans_datetime = datetime.strptime(trans_datetime, "%Y-%m-%d %H:%M:%S")
            
        tracker = BudgetTracker()
        tracker.add_transaction(account_type, amount, category, trans_datetime, "Income", notes)
        return gr.Info("Income transaction added successfully!")
    except Exception as e:
        return gr.Error(f"Error adding income transaction: {str(e)}")

def add_expense_transaction(account_type, amount, category, trans_datetime, notes):
    try:
        # Convert timestamp to MySQL datetime format
        if isinstance(trans_datetime, (int, float)):
            trans_datetime = datetime.fromtimestamp(trans_datetime).strftime('%Y-%m-%d %H:%M:%S')
        elif isinstance(trans_datetime, str):
            trans_datetime = datetime.strptime(trans_datetime, "%Y-%m-%d %H:%M:%S")
        tracker = BudgetTracker()
        tracker.add_transaction(account_type, amount, category, trans_datetime, "Expense", notes)
        return gr.Info("Expense transaction added successfully!")
    except Exception as e:
        return gr.Error(f"Error adding expense transaction: {str(e)}")

def add_income_budget_plan(estimated_amount, category, estimated_datetime, notes):
    try:
        tracker = BudgetTracker()
        tracker.add_budget("100", estimated_amount, category, estimated_datetime, "Income", notes)
        return gr.Info("Budget plan added successfully!")
    except Exception as e:
        return gr.Error(f"Error adding budget plan: {str(e)}")

def add_expense_budget_plan(estimated_percentage, category, estimated_datetime, notes):
    try:
        tracker = BudgetTracker()
        income_budget = tracker.get_monthly_income_budget()
        
        # Convert percentage to Decimal
        percentage = Decimal(str(estimated_percentage)) / Decimal(100)
        estimated_amount = percentage * income_budget  # Now both are Decimals
        
        tracker.add_budget(
            estimated_percentage=float(percentage * 100),  # Convert back to float for storage
            estimated_amount=estimated_amount,
            category=category,
            estimated_datetime=estimated_datetime,
            trans_type="Expense",
            notes=notes
        )
        return gr.Info(f"Budget plan added: {estimated_amount:,.2f}")
    except Exception as e:
        return gr.Error(f"Error adding budget plan: {str(e)}")

def show_weekly_report():
    try:
        tracker = BudgetTracker()
        report = tracker.get_weekly_report()
        
        if "error" in report:
            return gr.Error(report["error"])
        
        transactions_df = report["transactions"]
        summary_df = report["summary"]
        
        return transactions_df, summary_df
    except Exception as e:
        return gr.Error(f"Error generating report: {str(e)}")

def show_monthly_transaction_report():
    try:
        tracker = BudgetTracker()
        report = tracker.get_monthly_report()
        
        # Separate income and expense data
        income_data = [(row[0], row[1]) for row in report if row[2] == 'Income']
        expense_data = [(row[0], row[1]) for row in report if row[2] == 'Expense']
        
        # Generate charts
        income_chart = create_pie_chart(income_data, "Monthly Income Distribution") 
        expense_chart = create_pie_chart(expense_data, "Monthly Expense Distribution")
        
        return income_chart, expense_chart
    except Exception as e:
        return gr.Error(f"Error generating monthly report: {str(e)}")

def show_monthly_budget_tracker_report():
    try:
        tracker = BudgetTracker()
        
        # Generate charts
        estimated_comparison_data = tracker.get_monthly_estimated_comparison()
        estimated_comparison_chart = create_estimated_bar_chart(estimated_comparison_data)
        
        return estimated_comparison_chart  
    except Exception as e:
        return gr.Error(f"Error generating monthly report: {str(e)}")

In [55]:
# Gradio Interface
income_category = ['Allowance', 'Salary', 'Other']
expense_category = ['Accommodation', 'Apparel', 'Beauty', 'Education', 'Food', 'Household', 'Insurance', 'Social Life', 'Transport', 'Utility', 'Other']
account_type = ['Cash', 'Card']

with gr.Blocks() as app:
    gr.Markdown("# Student Budget Planning Tool")
    
    with gr.Tab("Income Transaction"):
        gr.Markdown("## Add Income Transaction")
        income_inputs = [
            gr.Dropdown(account_type, label="Account Type"),
            gr.Number(label="Amount"),
            gr.Dropdown(income_category, label="Category"),
            gr.DateTime(label="Transaction Date",type="datetime"),
            gr.Textbox(label="Notes")
        ]
        income_button = gr.Button("Add Income")
        income_output = gr.Textbox(label="Status")
        income_button.click(add_income_transaction, inputs=income_inputs, outputs=income_output)

    with gr.Tab("Expense Transaction"):
        gr.Markdown("## Add Expense Transaction")
        expense_inputs = [
            gr.Dropdown(account_type, label="Account Type"),
            gr.Number(label="Amount"),
            gr.Dropdown(expense_category, label="Category"),
            gr.DateTime(label="Transaction Date",type="datetime"),
            gr.Textbox(label="Notes")
        ]
        expense_button = gr.Button("Add Expense")
        expense_output = gr.Textbox(label="Status")
        expense_button.click(add_expense_transaction, inputs=expense_inputs, outputs=expense_output)

    with gr.Tab("Income Budget Planning"):
        gr.Markdown("## Add Income Budget Plan")
        budget_inputs = [
            gr.Number(label="Estimated Amount"),
            gr.Dropdown(income_category, label="Category"),
            gr.DateTime(label="Estimation Date",type="datetime"),
            gr.Textbox(label="Notes")
        ]
        budget_button = gr.Button("Add Budget Plan")
        budget_output = gr.Textbox(label="Status")
        budget_button.click(add_income_budget_plan, inputs=budget_inputs, outputs=budget_output)
    
    with gr.Tab("Expense Budget Planning"):
        gr.Markdown("## Add Expense Budget Plan")
        budget_inputs = [
            gr.Slider(1,100,label='Estimated Percentage'),
            gr.Dropdown(expense_category, label="Category"),
            gr.DateTime(label="Estimation Date",type="datetime"),
            gr.Textbox(label="Notes")
        ]
        budget_button = gr.Button("Add Budget Plan")
        budget_output = gr.Textbox(label="Status")
        budget_button.click(add_expense_budget_plan, inputs=budget_inputs, outputs=budget_output)

    with gr.Tab("Weekly Report"):
        gr.Markdown("## Weekly Transaction Report")
        weekly_button = gr.Button("Generate Weekly Report")
        weekly_table = gr.Dataframe(label="Weekly Transactions")
        weekly_summary = gr.Dataframe(label="Weekly Summary")
        weekly_button.click(
            show_weekly_report,
            inputs=None,
            outputs=[weekly_table, weekly_summary]
        )

    with gr.Tab("Monthly Transaction Report"):
        gr.Markdown("## Monthly Transaction Report")
        monthly_button = gr.Button("Generate Monthly Transaction Report")
        income_chart_output = gr.Plot()
        expense_chart_output = gr.Plot()
        monthly_button.click(show_monthly_transaction_report, inputs=None, outputs=[income_chart_output, expense_chart_output])

    with gr.Tab("Monthly Budget Tracker Report"):
        gr.Markdown("## Monthly Budget Tracker Report")
        monthly_button = gr.Button("Generate Monthly Budget Tracker Report")
        estimated_comparison_chart_output = gr.Plot()
        monthly_button.click(show_monthly_budget_tracker_report, inputs=None, outputs=estimated_comparison_chart_output) #outputs=[comparison_chart_output,estimated_comparison_chart_output])


if __name__ == "__main__":
    app.launch(share=True)


* Running on local URL:  http://127.0.0.1:7896
* Running on public URL: https://9d3254ab796a20b70f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
